## Podcast Audio

https://www.datacamp.com/tutorial/amazon-polly

In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent

In [2]:
model = init_chat_model(
    "gemini-2.0-flash", model_provider="google_genai")

# .with_structured_output(MyOutput)
response = model.invoke("Hello World!!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--faeda2d3-7cbe-417d-99bd-1ab0519bbeed-0' usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}}


In [4]:
# Student profile type definition
class StudentProfile(TypedDict):
    gender: str
    class_level: str  # e.g., "class 6", "12th", "undergrad", "postgrad"
    language: str     # e.g., "hindi", "english", "marathi"


class UserPrompt(TypedDict):
    topic: str
    pdf_file: str
    audio_file: str

# Graph state


class State(TypedDict):
    student_profile: StudentProfile
    user_prompt: UserPrompt
    summary_notes: str
    podcast_script: str
    mindmap: str
    quiz: str
    recommended_resources: str
    study_plan: str
    combined_output: str

In [31]:
# Prompts Templates
class PersonalizedPromptsStudent:
    def __init__(self, topic, student_profile: dict, user_instructions: str = ""):
        self.topic = topic
        self.student_profile = student_profile
        self.user_instructions = user_instructions

    def recommendation_resources(self):
        """Create a personalized prompt based on student profile"""

        prompt_template = ChatPromptTemplate([
           ("system", """
             You are an **expert academic female tutor and podcast content creator**. Your goal is to generate engaging, informative, and personalized educational podcast summaries for students.

            **Student Profile:**
            * **Class Level:** {class_level}
            * **Target Language:** {language} (Provide content in this language.)
            * **Preferred Pronouns:** {gender} (e.g., she/her, he/him, they/them – use to personalize direct address, if appropriate for the content style.)

            **Podcast Content Requirements:**
            1.  **Comprehensive Summary:** Generate a detailed and well-structured summary of the user-provided topic. This summary should serve as the core script for a **7-10 minute audio podcast episode within a 3000 character limit**.
            2.  **Academic Appropriateness:** Tailor the depth, complexity, and vocabulary of the content precisely to the specified **{class_level}**. Assume the student has foundational knowledge typical for their level, but introduce new concepts clearly.
            3.  **Engaging Delivery Style:**
                * Write in a conversational, accessible, and enthusiastic tone.
                * Incorporate brief, relatable examples or analogies where helpful.
                * Suggest pauses, changes in speaking rate, or emphasis using SSML tags to enhance listening experience (e.g., `<break>`, `<prosody>`).
                * Include a brief, friendly introduction and conclusion suitable for a podcast.
            4.  **SSML Formatting:** The entire response **must be formatted in SSML (Speech Synthesis Markup Language)**.
            5.  **Podcast Episode Structure:** Your summary should implicitly or explicitly follow a logical podcast flow:
                * **Introduction:** Hook the listener, introduce the topic.
                * **Main Content:** Break down the topic into digestible segments.
                * **Key Takeaways/Recap:** Briefly summarize the main points.
                * **Call to Action/Further Exploration:** Encourage continued learning.

            **Additional Instructions/Specific Focus:** {user_instructions} (e.g., "Focus on historical context," "Explain practical applications," "Include a quick quiz question at the end.")"""),

            ("user", "Create a podcast episode summary for the topic: **{topic}**. Also, provide a short list of **3-5 recommended resources** for further learning on this topic, mentioned at the end of the SSML content. Make sure to embed these recommendations clearly within the SSML as if spoken by the podcast host.")
        ])

        return prompt_template.invoke({
            "topic": self.topic,
            "class_level": self.student_profile.get("class_level", "general"),
            "language": self.student_profile.get("language", "English"),
            "gender": self.student_profile.get("gender", ""),
            "word_limit": "150",
            "user_instructions": self.user_instructions or "Focus on key concepts with examples"
        })

In [17]:
class PodcastContent(BaseModel):
    topic: str = Field(description="topic of the podcast")
    SSML: str = Field(description="SSML content for the podcast")
    

In [32]:
def node_recommendation_sources(state: State):
    """LLM call to generate recommendation materials"""

    prompt_instance = PersonalizedPromptsStudent(
        state['user_prompt']['topic'], state['student_profile'])

    prompt_message = prompt_instance.recommendation_resources()
        
    model = init_chat_model(
        "gemini-2.0-flash", model_provider="google_genai").with_structured_output(PodcastContent)

    response = model.invoke(prompt_message)
    
    print(response)

    return {"response": response}

In [13]:
input_state = {
    "student_profile": {
        "gender": "MALE",
        "class_level": "12th",
        "language": "HINDI"
    },
    "user_prompt": {
        "topic": "Chola kingdom"
    }
}

In [33]:
output = node_recommendation_sources(input_state)

topic='Chola kingdom' SSML='<speak>\n <prosody rate="medium">नमस्ते दोस्तों! आपका स्वागत है आज के इतिहास के रंग में रंगे पॉडकास्ट में। आज हम बात करेंगे चोल साम्राज्य की, जो दक्षिण भारत के इतिहास का एक बहुत ही महत्वपूर्ण हिस्सा है। तो चलिए, बिना किसी देरी के, शुरू करते हैं!</prosody>\n\n<break time="500ms"/>\n<prosody rate="slow">चोल साम्राज्य, जो लगभग 9वीं शताब्दी से 13वीं शताब्दी तक सत्ता में रहा, ने दक्षिण भारत को एक शक्तिशाली और समृद्ध साम्राज्य बनाया।</prosody> <prosody rate="medium">इनका साम्राज्य तमिलनाडु से लेकर श्रीलंका और मालदीव तक फैला हुआ था। चोल राजाओं ने एक मजबूत नौसेना बनाई, जिसकी वजह से वे समुद्री व्यापार में भी आगे रहे।</prosody>\n\n<break time="500ms"/>\n<prosody rate="medium">चोल साम्राज्य की सबसे खास बात थी उनका स्थानीय स्वशासन। उन्होंने गांवों में \'उर\' और \'सभा\' जैसी संस्थाएं बनाईं, जो गांव के मामलों को खुद देखती थीं।</prosody> <prosody rate="medium">इसके अलावा, चोल राजाओं ने कला और साहित्य को भी बहुत बढ़ावा दिया। उन्होंने बड़े-बड़े मंदिर बनवाए, जैसे कि तंजावुर क

In [35]:
output["response"].model_dump()

{'topic': 'Chola kingdom',
 'SSML': '<speak>\n <prosody rate="medium">नमस्ते दोस्तों! आपका स्वागत है आज के इतिहास के रंग में रंगे पॉडकास्ट में। आज हम बात करेंगे चोल साम्राज्य की, जो दक्षिण भारत के इतिहास का एक बहुत ही महत्वपूर्ण हिस्सा है। तो चलिए, बिना किसी देरी के, शुरू करते हैं!</prosody>\n\n<break time="500ms"/>\n<prosody rate="slow">चोल साम्राज्य, जो लगभग 9वीं शताब्दी से 13वीं शताब्दी तक सत्ता में रहा, ने दक्षिण भारत को एक शक्तिशाली और समृद्ध साम्राज्य बनाया।</prosody> <prosody rate="medium">इनका साम्राज्य तमिलनाडु से लेकर श्रीलंका और मालदीव तक फैला हुआ था। चोल राजाओं ने एक मजबूत नौसेना बनाई, जिसकी वजह से वे समुद्री व्यापार में भी आगे रहे।</prosody>\n\n<break time="500ms"/>\n<prosody rate="medium">चोल साम्राज्य की सबसे खास बात थी उनका स्थानीय स्वशासन। उन्होंने गांवों में \'उर\' और \'सभा\' जैसी संस्थाएं बनाईं, जो गांव के मामलों को खुद देखती थीं।</prosody> <prosody rate="medium">इसके अलावा, चोल राजाओं ने कला और साहित्य को भी बहुत बढ़ावा दिया। उन्होंने बड़े-बड़े मंदिर बनवाए, जैसे कि 

In [36]:
dict_response = output["response"].model_dump()
SSML = dict_response['SSML']
SSML

'<speak>\n <prosody rate="medium">नमस्ते दोस्तों! आपका स्वागत है आज के इतिहास के रंग में रंगे पॉडकास्ट में। आज हम बात करेंगे चोल साम्राज्य की, जो दक्षिण भारत के इतिहास का एक बहुत ही महत्वपूर्ण हिस्सा है। तो चलिए, बिना किसी देरी के, शुरू करते हैं!</prosody>\n\n<break time="500ms"/>\n<prosody rate="slow">चोल साम्राज्य, जो लगभग 9वीं शताब्दी से 13वीं शताब्दी तक सत्ता में रहा, ने दक्षिण भारत को एक शक्तिशाली और समृद्ध साम्राज्य बनाया।</prosody> <prosody rate="medium">इनका साम्राज्य तमिलनाडु से लेकर श्रीलंका और मालदीव तक फैला हुआ था। चोल राजाओं ने एक मजबूत नौसेना बनाई, जिसकी वजह से वे समुद्री व्यापार में भी आगे रहे।</prosody>\n\n<break time="500ms"/>\n<prosody rate="medium">चोल साम्राज्य की सबसे खास बात थी उनका स्थानीय स्वशासन। उन्होंने गांवों में \'उर\' और \'सभा\' जैसी संस्थाएं बनाईं, जो गांव के मामलों को खुद देखती थीं।</prosody> <prosody rate="medium">इसके अलावा, चोल राजाओं ने कला और साहित्य को भी बहुत बढ़ावा दिया। उन्होंने बड़े-बड़े मंदिर बनवाए, जैसे कि तंजावुर का बृहदेश्वर मंदिर, जो आज भी

In [37]:
import re

def clean_podcast_content(content):
    clean_text = re.sub(r"[\n\r]", "", content)
    return clean_text

SSML = clean_podcast_content(SSML)

SSML

'<speak> <prosody rate="medium">नमस्ते दोस्तों! आपका स्वागत है आज के इतिहास के रंग में रंगे पॉडकास्ट में। आज हम बात करेंगे चोल साम्राज्य की, जो दक्षिण भारत के इतिहास का एक बहुत ही महत्वपूर्ण हिस्सा है। तो चलिए, बिना किसी देरी के, शुरू करते हैं!</prosody><break time="500ms"/><prosody rate="slow">चोल साम्राज्य, जो लगभग 9वीं शताब्दी से 13वीं शताब्दी तक सत्ता में रहा, ने दक्षिण भारत को एक शक्तिशाली और समृद्ध साम्राज्य बनाया।</prosody> <prosody rate="medium">इनका साम्राज्य तमिलनाडु से लेकर श्रीलंका और मालदीव तक फैला हुआ था। चोल राजाओं ने एक मजबूत नौसेना बनाई, जिसकी वजह से वे समुद्री व्यापार में भी आगे रहे।</prosody><break time="500ms"/><prosody rate="medium">चोल साम्राज्य की सबसे खास बात थी उनका स्थानीय स्वशासन। उन्होंने गांवों में \'उर\' और \'सभा\' जैसी संस्थाएं बनाईं, जो गांव के मामलों को खुद देखती थीं।</prosody> <prosody rate="medium">इसके अलावा, चोल राजाओं ने कला और साहित्य को भी बहुत बढ़ावा दिया। उन्होंने बड़े-बड़े मंदिर बनवाए, जैसे कि तंजावुर का बृहदेश्वर मंदिर, जो आज भी उनकी महानता क

## Generate Podcast Audio using AWS

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/polly/client/synthesize_speech.html#

In [8]:
import boto3

polly = boto3.client('polly')

In [9]:
text_input = "<speak>नमस्ते! मेरा नाम काजल। मैं आपके द्वारा यहां टाइप किए गए किसी भी वाक्य को पढ़ सकती हूं।</speak>"

In [12]:
text_input = '<speak><p>नमस्ते! आज हम AI में अनिश्चितता के बारे में बात करेंगे।</p><s>यह मॉड्यूल AI सिस्टम्स में अनिश्चितता को मापने और प्रबंधित करने के तरीकों पर केंद्रित है।</s><break time="0.5s"/><p>AI एजेंटों को अनिश्चितता से निपटने की आवश्यकता होती है क्योंकि वे हमेशा यह नहीं जानते कि वे किस स्थिति में हैं या कार्यों के बाद कहां पहुंचेंगे।</p><s>वे belief state का ट्रैक रखकर अनिश्चितता को संभालते हैं।</s><break time="0.3s"/><p>उदाहरण के लिए, एक ऑटोमेटेड टैक्सी को हवाई अड्डे पर समय पर यात्री को पहुंचाने का लक्ष्य है, लेकिन उसे रास्ते में कई अनिश्चितताओं का सामना करना पड़ता है।</p><break time="0.5s"/><p>अनिश्चित reasoning का एक उदाहरण dental diagnosis है, जिसमें हमेशा अनिश्चितता शामिल होती है।</p><s>अनिश्चितता से निपटने के लिए probability theory सबसे अच्छा उपकरण है।</s><break time="0.3s"/><p>Diagnosis के समय, वास्तविक दुनिया में कोई अनिश्चितता नहीं होती: रोगी को या तो cavity होती है या नहीं।</p><break time="0.5s"/><p>सफलता दरों की तुलना करना, reliability का महत्व, trade-offs, preferences और outcomes, utility theory, और Maximum Expected Utility (MEU) का सिद्धांत महत्वपूर्ण हैं।</p><break time="0.5s"/><p>Probability Assertions में Random variables, domain, probability distribution, joint probability distribution, और full joint probability distribution महत्वपूर्ण हैं।</p><break time="0.5s"/><p>Probability Axioms में Non-negativity axiom और normalization axiom शामिल हैं।</p><break time="0.5s"/><p>Probabilistic inference अनिश्चितता में reasoning और decision-making की प्रक्रिया है।</p><s>इसमें समस्या को define करना, model बनाना, inference करना, और परिणामों का मूल्यांकन करना शामिल है।</s><break time="0.5s"/><p>Independence का मतलब है कि variables एक दूसरे को प्रभावित नहीं करते हैं।</p><s>Marginal independence variables को अलग-अलग मानना है, जबकि absolute independence का मतलब है कि variables एक दूसरे पर condition होने पर भी independent होते हैं।</s><break time="0.5s"/><p>Bayes\' Rule probabilistic inference के लिए fundamental है।</p><s>उदाहरण के लिए, spam filtering और medical diagnosis में इसका उपयोग होता है।</s><break time="0.5s"/><p>संक्षेप में, यह मॉड्यूल AI में अनिश्चितता की अवधारणा और इससे निपटने के विभिन्न तरीकों की गहरी समझ प्रदान करता है।</p><s>आगे जानने के लिए उत्सुक रहें!</s></speak>'

In [13]:
response = polly.synthesize_speech(
    Engine='neural',
    Text=text_input,
    TextType='ssml',
    LanguageCode='en-IN',  # 'hi-IN'
    OutputFormat='mp3',
    VoiceId='Kajal'
)

with open('speech.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())